# 7. CIFAR10 – transfer

In [2]:
import torch
import torchvision
import matplotlib.pyplot as plt
import pandas as pd
from fastai.data.all import *
from fastai.vision.all import * 

# 7. CIFAR10 – transfer

## A. `dls` 만들자

`-` X,y를 얻자.

In [4]:
path = untar_data(URLs.CIFAR)
path.ls()

In [25]:
labels = [str(l).split('/')[-1] for l in Path('/home/cgb3/.fastai/data/cifar10/train').ls()]
labels

In [53]:
X = torch.stack([torchvision.io.read_image(str(fname)) for l in labels for fname in Path(f'/home/cgb3/.fastai/data/cifar10/train/{l}').ls()],axis=0)/255
XX = torch.stack([torchvision.io.read_image(str(fname)) for l in labels for fname in Path(f'/home/cgb3/.fastai/data/cifar10/test/{l}').ls()],axis=0)/255
y = torch.tensor([i for i,l in enumerate(labels) for fname in Path(f'/home/cgb3/.fastai/data/cifar10/train/{l}').ls()])
yy = torch.tensor([i for i,l in enumerate(labels) for fname in Path(f'/home/cgb3/.fastai/data/cifar10/test/{l}').ls()])

`-` 데이터를 시각화해보자.

`-` dls를 만들자.

In [63]:
ds1 = torch.utils.data.TensorDataset(X,y)
dl1 = torch.utils.data.DataLoader(ds1)
ds2 = torch.utils.data.TensorDataset(XX,yy)
dl2 = torch.utils.data.DataLoader(ds2)
dls = DataLoaders(dl1,dl2)

`-` 아래와 같이 쉽게 만들수도있음…

In [62]:
# dls = ImageDataLoaders.from_folder(
#     path= path,
#     train= 'train',
#     valid= 'test'
# )

## B. 수제네트워크로 학습

In [64]:
XX.shape

`-` 시도1:

In [65]:
y

In [71]:
# step1: 데이터의 정리
ds1 = torch.utils.data.TensorDataset(X,y)
dl1 = torch.utils.data.DataLoader(ds1,batch_size=128,shuffle=True)
ds2 = torch.utils.data.TensorDataset(XX,yy)
dl2 = torch.utils.data.DataLoader(ds2,batch_size=10000)
dls = DataLoaders(dl1,dl2)
# step2: 학습에 필요한 오브젝트들을 선언 
torch.manual_seed(43052)
net = torch.nn.Sequential(
    torch.nn.Conv2d(3,16,kernel_size=(2,2)),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=(2,2)),
    torch.nn.Flatten(),
    torch.nn.Linear(3600,10)
)
loss_fn = torch.nn.CrossEntropyLoss()
lrnr = Learner(
    dls = dls,
    model = net,
    loss_func = loss_fn,
    metrics = [accuracy],
    #---#
)
# step3: 학습을 하는 과정 = 적합을 하는 과정 
lrnr.fit(10)
## step4: 
lrnr.model.to("cpu")
print(f'train = {(lrnr.model(X).argmax(axis=-1) == y).float().mean():.4f}')
print(f'val = {(lrnr.model(XX).argmax(axis=-1) == yy).float().mean():.4f}')

train = 0.6167
val = 0.5754

`-` 시도2:

`-` 시도3:

In [74]:
# step1: 데이터의 정리
ds1 = torch.utils.data.TensorDataset(X,y)
dl1 = torch.utils.data.DataLoader(ds1,batch_size=128,shuffle=True)
ds2 = torch.utils.data.TensorDataset(XX,yy)
dl2 = torch.utils.data.DataLoader(ds2,batch_size=10000)
dls = DataLoaders(dl1,dl2)
# step2: 학습에 필요한 오브젝트들을 선언 
torch.manual_seed(43052)
net = torch.nn.Sequential(
    torch.nn.Conv2d(3,128,kernel_size=(2,2)),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=(2,2)),    
    torch.nn.Conv2d(128,128,kernel_size=(2,2)),
    torch.nn.ReLU(),    
    torch.nn.MaxPool2d(kernel_size=(2,2)),
    torch.nn.Flatten(),
    torch.nn.Linear(6272,800),
    torch.nn.ReLU(),
    torch.nn.Linear(800,10)    
)
loss_fn = torch.nn.CrossEntropyLoss()
lrnr = Learner(
    dls = dls,
    model = net,
    loss_func = loss_fn,
    metrics = [accuracy],
    #---#
)
# step3: 학습을 하는 과정 = 적합을 하는 과정 
lrnr.fit(10)
## step4: 
lrnr.model.to("cpu")
print(f'train = {(lrnr.model(X).argmax(axis=-1) == y).float().mean():.4f}')
print(f'val = {(lrnr.model(XX).argmax(axis=-1) == yy).float().mean():.4f}')

train = 0.9144
val = 0.7067

## C. TransferLearning으로 학습

`-` ResNet18을 다운로드:

In [76]:
net = torchvision.models.resnet18()
net 

-   `torchvision.models.resnet18()`: resnet18이라는 네크워크 대신
    선언해주는 함수..

`-` 마지막의 레이어만 수정

In [80]:
net.fc = torch.nn.Linear(512,10)

`-` 학습해보자.

In [82]:
# step1: 데이터의 정리
ds1 = torch.utils.data.TensorDataset(X,y)
dl1 = torch.utils.data.DataLoader(ds1,batch_size=128,shuffle=True)
ds2 = torch.utils.data.TensorDataset(XX,yy)
dl2 = torch.utils.data.DataLoader(ds2,batch_size=10000)
dls = DataLoaders(dl1,dl2)
# step2: 학습에 필요한 오브젝트들을 선언 
net = torchvision.models.resnet18()
net.fc = torch.nn.Linear(512,10)
loss_fn = torch.nn.CrossEntropyLoss()
lrnr = Learner(
    dls = dls,
    model = net,
    loss_func = loss_fn,
    metrics = [accuracy],
    #---#
)
# step3: 학습을 하는 과정 = 적합을 하는 과정 
lrnr.fit(10)
## step4: 
lrnr.model.to("cpu")
print(f'train = {(lrnr.model(X).argmax(axis=-1) == y).float().mean():.4f}')
print(f'val = {(lrnr.model(XX).argmax(axis=-1) == yy).float().mean():.4f}')

train = 0.9123
val = 0.7322